# Figure Out How To Use do-mpc Package

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import casadi as cas
import do_mpc

from cas_models.continuous_time.models import StateSpaceModelCT
from cas_models.transformations import connect_systems
from feed_conc_ctrl import MixingTankModelCT, FlowMixerCT

## Define CasADi System Model with this Library

```none
          ┌────────┐        
     ────►┤ Tank 1 │        
          │        │        
          │        ├──┐     
          └────────┘  │     
                      │     
                      ├────►
                      │     
          ┌────────┐  │     
     ────►┤ Tank 2 │  │     
          │        │  │     
          │        ├──┘     
          └────────┘    
```

In [ ]:
# Create mixing tank model 
tank_1_model = MixingTankModelCT(D=5, name='tank_1')
tank_2_model = MixingTankModelCT(D=5, name='tank_2')
mixer_model = FlowMixerCT(2, name='mixer')
systems = [tank_1_model, tank_2_model, mixer_model]

def print_sys_dimensions(sys):
    print(sys.name, f"({sys.ny}x{sys.nu})")
    for attr_name in ["input_names", "state_names", "output_names"]:
        print(f"{attr_name:>15s}: {getattr(sys, attr_name)}")

for sys in systems:
    print_sys_dimensions(sys)

In [ ]:
# Connect systems together to form system of 2 tanks and a mixer
model_class = StateSpaceModelCT
connections = [
    ('tank_1_v_dot_out', 'mixer_v_dot_in_1'),
    ('tank_1_conc_out', 'mixer_conc_in_1'),
    ('tank_2_v_dot_out', 'mixer_v_dot_in_2'),
    ('tank_2_conc_out', 'mixer_conc_in_2'),
]

tanks_system = connect_systems(
    systems, connections, model_class, name="tanks_system", verbose_names=True
)

print_sys_dimensions(tanks_system)

## Define MPC controller

Do MPC variable types

  | Long name | Short name | Remark   |
  |-----------|------------|----------|
  | states    | `_x`       | Required |
  | inputs    | `_u`       | Required |
  | algebraic | `_z`       | Optional |
  | parameter | `_p`       |          |

In [ ]:
tanks_system.state_names

In [ ]:
# Initialize new system model
mpc_model_type = 'continuous'
mpc_model = do_mpc.model.Model(mpc_model_type)

# Define MPC model state variables same as tanks system
mpc_state_vars = [
    mpc_model.set_variable(var_type='_x', var_name=name, shape=(1, 1))
    for name in tanks_system.state_names
]

mpc_state_vars

In [ ]:
# Input disturbance - assume constants for now
input_disturbances = {
    'tank_1_conc_in': 0.0,
    'tank_2_conc_in': 1.0,
}

# Manipulated variables
mv_names = [
    'tank_1_v_dot_in',
    'tank_1_v_dot_out',
    'tank_2_v_dot_in',
    'tank_2_v_dot_out'
]

# Define MPC model mvs
mpc_mvs = [
    mpc_model.set_variable(var_type='_u', var_name=name, shape=(1, 1))
    for name in mv_names
]
mpc_mvs

In [ ]:
# Another way to retrieve state and input variables
print("States:", mpc_model.x[tanks_system.state_names])
print("MVs:", mpc_model.u[mv_names])

In [ ]:
# Check signature of tank system model functions
tanks_system.f

In [ ]:
u = []
for name in tanks_system.input_names:
    if name in mv_names:
        u.append(mpc_model.u[name])
    elif name in input_disturbances:
        u.append(input_disturbances[name])
u = cas.vcat(u)
u

In [ ]:
t = 0.0  # time variable is not used in this model
rhs = tanks_system.f(t, mpc_model.x, u)
rhs

In [ ]:
for i, name in enumerate(mpc_model.x.keys()):
    mpc_model.set_rhs(name, rhs[i])


In [ ]:
# The tanks_system has an output function h(t, x, u)
y = tanks_system.h(t, mpc_model.x, u)

# Register each output as a measurement and define measurement equations
for i, name in enumerate(tanks_system.output_names):
    mpc_model.set_meas(name, y[i], meas_noise=True)

In [ ]:
mpc_model.setup()

## Define State Estimator

In [ ]:
# Option 1: Simple StateFeedback (assumes perfect measurements)
# estimator = do_mpc.estimator.StateFeedback(mpc_model)

# Option 2: Extended Kalman Filter (handles noise)
estimator = do_mpc.estimator.EKF(mpc_model)
estimator.P0 = 100.0 * np.eye(mpc_model.n_x)  # Initial covariance
estimator.setup()

In [ ]:
np.eye(mpc_model.n_x)

## Configure the MPC controller

In [ ]:
mpc = do_mpc.controller.MPC(mpc_model)
mpc

In [ ]:
setup_mpc = {
    'n_horizon': 20,
    't_step': 1.0,
    'n_robust': 1,
    'store_full_solution': True,
}
mpc.set_param(**setup_mpc)

In [ ]:
mpc_model.x.keys()

In [ ]:
mpc_model.x[]

In [ ]:
# Objective function
mterm = phi_1**2 + phi_2**2 + phi_3**2
lterm = phi_1**2 + phi_2**2 + phi_3**2

mpc.set_objective(mterm=mterm, lterm=lterm)

In [ ]:
# Constraints

# Lower and upper bounds on states:
mpc.bounds['lower', '_x', 'L'] = 0.25
mpc.bounds['upper', '_x', 'L'] = 3.0

# Lower and upper inputs:
mpc.bounds['lower', '_u', 'phi_m_1_set'] = 0.0
mpc.bounds['upper', '_u', 'phi_m_2_set'] = 2*np.pi